In [ ]:
import requests
import json
import pandas as pd
import csv

Connecting to GraphQL

In [ ]:
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

dgraph_url = "https://blue-surf-570083.us-east-1.aws.cloud.dgraph.io/graphql" # this is your GraphQL Dgraph endpoint

transport = RequestsHTTPTransport(
    url=dgraph_url, verify=True, retries=3,
)

client = Client(transport=transport, fetch_schema_from_transport=True)

In [ ]:
df=pd.read_csv("data3.csv")#Reading File and creating Multiple Copies
df1= df
df2= df
df3=df
df3

In [ ]:
df3.columns #Displaying Columns name

In [ ]:
##Dropping Columns and storing it in a dataframe isomap
isomap=df3.drop(['english', 'category', 'cids', 'similar_word', 'probability', 'Mapping'],axis=1)

In [ ]:
#Reading Column Names and storing it as Label
label=isomap.columns
label

In [ ]:
#Separating the words in columns For Labels
di={}

for i in label:
    d1=pd.DataFrame()
    call1=[]
    
    try:
        d1=(isomap[i].str.split('/',expand=True))
        for z in range(len(d1.columns.to_list())):
            call1.append(str(i)+','+str(z))
        
        d1.columns=call1
        if len(ss)==0:
            ss=d1
        else:
            ss=pd.merge(d1,ss,how='inner',left_index=True,right_index=True)
    except:
        continue



In [ ]:
#Extracting ISO639 Code from File
isom=ss
lab=isom.columns
da=[]
for i in lab:
    c=i.split(',')[0]
    if c not in da:
        da.append(c)

In [ ]:
#Reading ISO639 and Language extracted from SIL Website and Joining to get Language with respect to IS0639
code= pd.read_csv('iso-639-3.tab', encoding='utf8',sep='\t')
mp=code.drop(['Part1','Part2B','Part2T','Scope','Language_Type','Comment'],axis=1)
da=pd.DataFrame(da)
da.columns=['Id']
daa=pd.merge(da,mp,on='Id')
daa

In [ ]:
#Separating the words in columns For Semantic Domian Loading
ess=pd.DataFrame()
di={}

for i in label:
    d1=pd.DataFrame()
    call1=[]
    try:
        d1=(df3[i].str.split('/',expand=True))
        for z in range(len(d1.columns.to_list())):
            call1.append(str(i)+','+str(z))
        
        d1.columns=call1
        if len(ess)==0:
            ess=d1
        else:
            ess=pd.merge(d1,ess,how='inner',left_index=True,right_index=True)
    except:
        continue

In [ ]:
#Adding columns from df2 dataframe to ess for final loading process
ess['cids']=df3['cids']
ess['eng']=df3['english']
ess['category']=df3['category']
ess['Mapping']=df3['Mapping']
ess['similar_word']=df3['similar_word']


Defining Function to mutate Semantic Domain to GraphQL

In [ ]:
def test_mutation_result2(client,params):
    query = gql(
        """
mutation Mutation($input: [AddSemanticDomainInput!]!) {
  addSemanticDomain(input: $input, upsert: true) {
    semanticDomain {
      cids
      name
      label
      words {
        text
        isoLanguage {
          iso639
        }
      }
    }
  }
}


        """
    )
    result = client.execute(query, variable_values=params)

In [ ]:
#Loading the data into GraphQL by calling function test_mutation_result2
i=0
while i<=len(df3):
    for k in ss:
        sso=k.split(',')[0]
        try:
            params={"input":{"cids":ess.loc[i,"cids"], "name":ess.loc[i,"category"],"label":ess.loc[i,"Mapping"],"words":{"text":ess.loc[i,k],"isoLanguage":{"iso639":sso}}}}
            test_mutation_result2(client,params)
        except:
            continue

    i=i+1